---

<div style="text-align: center; font-family: Arial, sans-serif; margin-top: 50px;">
    <h1 style="font-size: 36px; font-weight: bold;"><b>Prácticas de NLP</b></h1>
    <h2 style="font-size: 28px; color: #2E86C1;"><b>NLP con Long-Short Term Memory (LSTM)</b></h2>
    <p style="font-size: 20px; margin-top: 30px;">
        <b>Materia:</b> Procesamiento de Lenguaje Natural<br>
        <b>Estudiantes:</b> Albin Rivera y Yesid Castelblanco<br>
        <b>Fecha:</b> 23 de Agosto de 2025
    </p>
</div>

---

## Referencias
- Dataset: [Fake News Corpus Spanish](https://huggingface.co/mariagrandury/fake_news_corpus_spanish)  
- Librerías: Hugging Face `datasets`, Pandas, warnings  

# **1. Configuración Inicial**
---

<p style="font-size: 16px;">
En esta primera sección se realiza la configuración del entorno de trabajo. Se importan las librerías necesarias para el procesamiento de lenguaje natural (NLTK, Transformers, Datasets), el manejo de datos (NumPy, Pandas), y el entrenamiento de modelos de Deep Learning con PyTorch y PyTorch Lightning. Además, se preparan métricas de evaluación como exactitud, precisión, recall y F1-score, que serán utilizadas en el proceso de entrenamiento y validación del modelo.

In [1]:
import os
import warnings
import sys
import torch
import numpy as np
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, logging as hf_logging
from datasets import load_dataset
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torchmetrics.classification import Accuracy, Precision, Recall, F1Score
from torch import nn

<p style="font-size: 16px;">
Adicional, se configuran parámetros específicos para asegurar compatibilidad entre entornos como Colab o Kaggle, se ajusta el número de workers, se descargan las stopwords en español y se deshabilitan ciertas optimizaciones para evitar conflictos con librerías de GPU. También se instalan las dependencias necesarias y se suprimen advertencias innecesarias que podrían dificultar la lectura de resultados.

In [2]:
# Configuraciones para compatibilidad con Kaggle y Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
    num_workers = 0
else:
    num_workers = 4

# Descargar stopwords para español
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

# Deshabilitar oneDNN para evitar errores en LSTM
torch.backends.mkldnn.enabled = False
os.environ["ONEDNN_VERBOSE"] = "all"

# Deshabilitar XLA para evitar conflictos con cuFFT/cuDNN/cuBLAS
os.environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir=/usr/lib/cuda"

# Suprimir advertencias y configurar tokenización paralela
warnings.filterwarnings('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Instalar dependencias
print("📦 Instalando dependencias...")
!pip install torch==2.3.0 transformers==4.38.2 datasets==2.14.5 pytorch-lightning==2.2.1 torchmetrics==1.0.3 nlpaug==1.1.11 --quiet

# Configurar logging de Hugging Face
hf_logging.set_verbosity_error()

📦 Instalando dependencias...


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 968.0 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 102.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 2.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 99.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.

# **2. Preprocesamiento de textos**

<p style="font-size: 16px;">
Aqui se define una función para limpiar y normalizar el texto, convirtiéndolo a minúsculas, eliminando stopwords y conservando solo aquellos signos de puntuación que son relevantes en el idioma español (como exclamaciones e interrogaciones), permitiendo reducir el ruido en los datos y mejorar el rendimiento de los modelos posteriores.

In [3]:
def preprocess_text(text):
    """Limpia el texto: minúsculas, conserva puntuación relevante, elimina stopwords."""
    text = text.lower()
    # Conservar signos de exclamación e interrogación
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

# **3. Cargar y explorar el Dataset**

<p style="font-size: 16px;">
En esta parte se carga el dataset fake_news_corpus_spanish desde HuggingFace, se convierte en un DataFrame de Pandas y se aplican transformaciones iniciales como la conversión de la variable CATEGORY a valores binarios y el preprocesamiento de los textos. Además, se calculan estadísticas básicas de longitud de los textos y la distribución de las categorías, lo cual ayuda a detectar posibles desbalances en las clases.

In [4]:
def load_and_explore_dataset():
    """Carga el dataset y muestra estadísticas básicas."""
    print("📥 Cargando 'fake_news_corpus_spanish' dataset (split: test)...")
    try:
        dataset = load_dataset("mariagrandury/fake_news_corpus_spanish", split="test")
        df = dataset.to_pandas()
    except Exception as e:
        print(f"Error al cargar el dataset: {e}")
        sys.exit(1)

    # Convertir CATEGORY a binario y preprocesar textos
    df['CATEGORY'] = df['CATEGORY'].astype(int)
    df['TEXT'] = df['TEXT'].apply(preprocess_text)

    # Mostrar información del dataset
    print("\nColumnas del Dataset:", df.columns.tolist())
    print("\nPrimeras Filas:\n", df.head())
    print("\nDistribución de Categorías:\n", df['CATEGORY'].value_counts())
    print(f"\nTotal de Textos: {len(df)}")

    # Estadísticas de longitud de texto
    text_lengths = [len(text) for text in df['TEXT']]
    print(f"Texto más corto: {min(text_lengths)}")
    print(f"Texto más largo: {max(text_lengths)}")
    print(f"Longitud promedio: {np.mean(text_lengths):.2f}")

    return dataset


# **4. Dividir el Dataset**

<p style="font-size: 16px;">
Una vez cargado y explorado, el dataset se divide en subconjuntos para permitir un entrenamiento robusto y evaluaciones confiables. En esta sección se define una función que separa el dataset en tres particiones las cuales son entrenamiento (80%), validación (10%) y prueba (10%).

In [5]:
def split_dataset(dataset):
    """Divide el dataset en conjuntos de entrenamiento, validación y prueba."""
    dataset_size = len(dataset)
    train_size = int(0.8 * dataset_size)
    val_size = int(0.1 * dataset_size)
    test_size = dataset_size - train_size - val_size

    train_subset, val_subset, test_subset = random_split(
        dataset,
        lengths=[train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )

    print(f"✅ Train: {len(train_subset)}, Val: {len(val_subset)}, Test: {len(test_subset)}")
    return train_subset, val_subset, test_subset

# **5. Clase de Dataset personalizado**

<p style="font-size: 16px;">
En este punto se implementa una clase personalizada de Dataset para manejar el corpus de noticias falsas en español. La clase se encarga de tokenizar los textos, aplicar truncamiento y padding hasta una longitud máxima definida, y asociar cada texto con su etiqueta correspondiente.

In [6]:
class FakeNewsCorpusSpanishDataset(Dataset):
    """Dataset personalizado para clasificación de noticias falsas."""
    def __init__(self, tokenizer, dataset, seq_length=512):
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.seq_length = seq_length
        self.id_2_class_map = {0: 'False', 1: 'True'}
        self.class_2_id_map = {'False': 0, 'True': 1}
        self.num_classes = len(self.id_2_class_map)

    def __getitem__(self, index):
        """Tokeniza el texto preprocesado y retorna tensores."""
        text = preprocess_text(self.dataset[index]['TEXT'])
        label = self.dataset[index]['CATEGORY']
        label = self.class_2_id_map['True' if label == 1 else 'False']

        tokenized = self.tokenizer(
            text,
            max_length=self.seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'y': torch.tensor(label, dtype=torch.long)
        }

    def __len__(self):
        return len(self.dataset)


# **6. Definición del modelo LSTM**

<p style="font-size: 16px;">
Aquí se define el modelo de clasificación basado en una red recurrente LSTM (Long Short-Term Memory). Este modelo utiliza embeddings para representar las palabras, capas LSTM bidireccionales para capturar dependencias en ambas direcciones del texto, normalización por lotes para estabilizar el entrenamiento y capas totalmente conectadas con dropout para mejorar la generalización.

In [7]:
class LSTMClassifier(nn.Module):
    """Clasificador basado en LSTM para detección de noticias falsas."""
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2, dropout=0.5)
        self.batch_norm = nn.BatchNorm1d(hidden_dim * 2)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, input_ids):
        embedded = self.embedding(input_ids)
        lstm_out, _ = self.lstm(embedded)
        pooled = lstm_out[:, -1, :]
        pooled = self.batch_norm(pooled)
        pooled = self.dropout(pooled)
        logits = self.fc(pooled)
        return logits

# **7. Módulo de PyTorch Lightning**

<p style="font-size: 16px;">
Para simplificar el entrenamiento, validación y prueba del modelo, se crea un módulo basado en PyTorch Lightning. Esta clase integra el modelo LSTM con la función de pérdida, optimizador, programador de tasa de aprendizaje y métricas de evaluación. Además, incluye métodos específicos para cada fase del entrenamiento y predicción, asegurando un flujo estandarizado y reproducible.

In [8]:
class SpanishNewsClassifierWithLSTM(pl.LightningModule):
    """Módulo de PyTorch Lightning para entrenar el clasificador LSTM."""
    def __init__(self, vocab_size, num_classes, embed_dim=256, hidden_dim=128, lr=3e-4):
        super().__init__()
        self.save_hyperparameters()
        self.model = LSTMClassifier(vocab_size, embed_dim, hidden_dim, num_classes)
        self.loss_fn = nn.CrossEntropyLoss()

        # Inicializar pesos
        for module in self.model.modules():
            if isinstance(module, nn.LSTM):
                nn.init.xavier_uniform_(module.weight_ih_l0)
                nn.init.xavier_uniform_(module.weight_hh_l0)
                nn.init.xavier_uniform_(module.weight_ih_l1)
                nn.init.xavier_uniform_(module.weight_hh_l1)
            elif isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)

        # Métricas para clasificación binaria
        self.train_acc = Accuracy(task='binary', num_classes=num_classes)
        self.val_acc = Accuracy(task='binary', num_classes=num_classes)
        self.test_acc = Accuracy(task='binary', num_classes=num_classes)
        self.test_precision = Precision(task='binary', num_classes=num_classes)
        self.test_recall = Recall(task='binary', num_classes=num_classes)
        self.test_f1 = F1Score(task='binary', num_classes=num_classes)

    def forward(self, input_ids):
        return self.model(input_ids)

    def training_step(self, batch, batch_idx):
        x, y = batch['input_ids'], batch['y']
        logits = self(x)
        loss = self.loss_fn(logits, y)
        probs = logits.softmax(dim=-1)[:, 1]
        self.train_acc(probs, y)
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', self.train_acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch['input_ids'], batch['y']
        logits = self(x)
        loss = self.loss_fn(logits, y)
        probs = logits.softmax(dim=-1)[:, 1]
        self.val_acc(probs, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', self.val_acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch['input_ids'], batch['y']
        logits = self(x)
        probs = logits.softmax(dim=-1)[:, 1]
        self.test_acc(probs, y)
        self.test_precision(probs, y)
        self.test_recall(probs, y)
        self.test_f1(probs, y)
        self.log('test_acc', self.test_acc, prog_bar=True)
        self.log('test_precision', self.test_precision, prog_bar=True)
        self.log('test_recall', self.test_recall, prog_bar=True)
        self.log('test_f1', self.test_f1, prog_bar=True)

    def predict_step(self, batch, batch_idx):
        x = batch['input_ids']
        logits = self(x)
        probs = logits.softmax(dim=-1)
        return probs

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.lr, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {"scheduler": scheduler, "monitor": "val_loss"}
        }

    def on_train_batch_end(self, outputs, batch, batch_idx):
        torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=1.0)

# **8. Función de Predicción**

<p style="font-size: 16px;">
Con el modelo ya entrenado, esta función permite evaluar textos nuevos para determinar si son verdaderos o falsos. Se aplica el mismo preprocesamiento y tokenización que en el entrenamiento, y luego se obtiene la predicción junto con la probabilidad asociada.

In [9]:
def predict_text(model, tokenizer, text, seq_length=512, device='cuda' if torch.cuda.is_available() else 'cpu'):
    """Predice la clase de un texto nuevo."""
    model.eval()
    model.to(device)
    text = preprocess_text(text)
    tokenized = tokenizer(
        text,
        max_length=seq_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    input_ids = tokenized['input_ids'].to(device)
    with torch.no_grad():
        logits = model(input_ids)
        probs = logits.softmax(dim=-1)
        pred_class = torch.argmax(probs, dim=-1).item()

    id_2_class = {0: 'False', 1: 'True'}
    return id_2_class[pred_class], probs[0][pred_class].item()

# **9. Ejecución Principal y Metricas de Evaluación del Modelo**

<p style="font-size: 16px;">
Finalmente, se organiza todo el flujo en una función principal. Aquí se configuran los dispositivos de cómputo (CPU, GPU o TPU), se carga y divide el dataset, se inicializa el tokenizador y el modelo, y se definen los callbacks de entrenamiento como el EarlyStopping y la selección del mejor checkpoint. Tras el entrenamiento, el modelo se evalúa en el conjunto de prueba, se generan predicciones y se muestra un ejemplo de predicción aplicada a un texto real.

In [14]:
def main():
    """Función principal para ejecutar el pipeline."""
    # Verificar disponibilidad de GPU/TPU
    if IN_COLAB and torch.cuda.is_available():
        device = torch.device('cuda')
        print("CUDA Disponible: True")
        print("Nombre de GPU:", torch.cuda.get_device_name(0))
    elif IN_COLAB and 'XLA' in torch.__version__:
        device = torch.device('xla')
        print("TPU Disponible: True")
    else:
        device = torch.device('cpu')
        print("CUDA/TPU no disponible, usando CPU")

    # Cargar dataset
    dataset = load_and_explore_dataset()

    # Inicializar tokenizador
    try:
        tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
    except Exception as e:
        print(f"Error al cargar el tokenizador: {e}")
        sys.exit(1)

    # Dividir dataset y almacenar índices
    train_subset, val_subset, test_subset = split_dataset(dataset)
    test_indices = test_subset.indices

    # Crear datasets personalizados
    train_dataset = FakeNewsCorpusSpanishDataset(tokenizer, train_subset, seq_length=512)
    val_dataset = FakeNewsCorpusSpanishDataset(tokenizer, val_subset, seq_length=512)
    test_dataset = FakeNewsCorpusSpanishDataset(tokenizer, test_subset, seq_length=512)

    # Crear dataloaders
    batch_size = 16  # Reducir para gradientes más precisos
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True)

    # Inicializar modelo
    model = SpanishNewsClassifierWithLSTM(
        vocab_size=tokenizer.vocab_size,
        num_classes=train_dataset.num_classes,
        embed_dim=256,
        hidden_dim=128,
        lr=3e-4
    )

    # Configurar logger y callbacks
    tb_logger = TensorBoardLogger('tb_logs', name='LSTMClassifier')
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, mode='min'),
        ModelCheckpoint(monitor='val_acc', mode='max', save_top_k=1, filename='best-checkpoint', dirpath='checkpoints')
    ]

    # Inicializar entrenador
    trainer = pl.Trainer(
        max_epochs=20,
        accelerator="gpu" if torch.cuda.is_available() else "tpu" if IN_COLAB and 'XLA' in torch.__version__ else "cpu",
        devices=1,
        logger=tb_logger,
        callbacks=callbacks,
        precision="16-mixed" if torch.cuda.is_available() else "16-true" if IN_COLAB and 'XLA' in torch.__version__ else "32-true",
        num_sanity_val_steps=0
    )

    # Entrenar modelo
    print("🚀 Iniciando entrenamiento...")
    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

    # Evaluar en conjunto de prueba
    print("🔎 Evaluando en conjunto de prueba...")
    trainer.test(model, dataloaders=test_loader)

    # Generar DataFrame con predicciones
    print("📊 Generando DataFrame con predicciones...")
    model.eval()
    predictions = trainer.predict(model, test_loader)
    predictions = torch.cat(predictions, dim=0)
    predictions = torch.argmax(predictions, dim=-1)
    predictions = [train_dataset.id_2_class_map[pred.item()] for pred in predictions]

    df = pd.DataFrame(data={
        "texto": [dataset[i]['TEXT'] for i in test_indices],
        "categoría": [train_dataset.id_2_class_map[dataset[i]['CATEGORY']] for i in test_indices],
        "predicción": predictions[:len(test_indices)]
    }, index=test_indices)
    print(df.head(15))

    # Ejemplo de predicción
    sample_text = "El presidente anunció nuevas medidas económicas."
    prediction, confidence = predict_text(model, tokenizer, sample_text)
    print(f"\n📝 Predicción de Ejemplo: '{sample_text}' -> {prediction} (Confianza: {confidence:.4f})")

if __name__ == "__main__":
    main()

CUDA Disponible: True
Nombre de GPU: Tesla P100-PCIE-16GB
📥 Cargando 'fake_news_corpus_spanish' dataset (split: test)...

Columnas del Dataset: ['ID', 'CATEGORY', 'TOPICS', 'SOURCE', 'HEADLINE', 'TEXT', 'LINK']

Primeras Filas:
    ID  CATEGORY    TOPICS         SOURCE  \
0   1         1  Covid-19  El Economista   
1   2         0  Política     El matinal   
2   3         1  Política        El País   
3   4         0  Política     AFPFactual   
4   5         1  Sociedad   La Republica   

                                            HEADLINE  \
0                       Covid-19: mentiras que matan   
1  El Gobierno podrá acceder a las IPs de los móv...   
2  La comunidad musulmana catalana denuncia a Vox...   
3                                               None   
4  El censo poblacional 2018 tendrá un costo de $...   

                                                TEXT  \
0  control covid-19 sólo tema médicos resto perso...   
1  gobierno pedro sánchez pablo iglesias encontra...   
2

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7291666865348816     │
│          test_f1          │    0.6285714507102966     │
│      test_precision       │    0.7333333492279053     │
│        test_recall        │     0.550000011920929     │
└───────────────────────────┴───────────────────────────┘

📊 Generando DataFrame con predicciones...


Predicting: |          | 0/? [00:00<?, ?it/s]

                                                 texto categoría predicción
296  Las vacunas contra la COVID-19 -13 de ellas en...      True      False
336  Si eres de los que ama tomarse una copita de v...      True      False
99   Así lo reveló el estudio en el que participaro...      True       True
516  Vacaciones de Semana Santa, buen tiempo, el ha...      True       True
227  Varios vuelos con salida de Casablanca y desti...      True      False
344  Se está haciendo viral tras recuperarse una gr...     False      False
74   MASCARILLAS.... PREFERENTEMENTE PARA CARNAVAL ...     False      False
207  El Laboratorio Biológico Chino de Wuhan es en ...     False      False
480  El Papa Francisco podría estar enfermo de Coro...     False      False
325  El expresidente del Gobierno Felipe González h...     False       True
174  Una veintena de jóvenes del entorno abertzale ...     False      False
431  El embarazo es parte de nuestra sexualidad y v...      True       True
198  FUE TAN

# **10. Resultados**

<p style="font-size: 16px;">
1. Los resultados obtenidos en la evaluación del modelo muestran un desempeño general aceptable, alcanzando una exactitud del 72.91%. Este valor indica que, en promedio, el modelo clasifica correctamente cerca de tres cuartas partes de los textos evaluados. La precisión alcanzó un valor de 0.73, lo que evidencia que, cuando el sistema identifica una noticia como verdadera o falsa, en la mayoría de los casos acierta en dicha clasificación. 

<p style="font-size: 16px;">
2. Sin embargo, el recall obtenido fue de 0.55, lo que revela que el modelo no logra identificar de manera efectiva todas las noticias falsas presentes en el conjunto de prueba, dejando escapar una proporción significativa de estas.

<p style="font-size: 16px;">
3. El valor del F1-score, situado en 0.62, refleja un equilibrio moderado entre la precisión y el recall, pero también confirma la necesidad de fortalecer la sensibilidad del modelo para detectar noticias falsas sin sacrificar en exceso la precisión alcanzada. Al observar los ejemplos en el DataFrame generado, se aprecia que, aunque en muchos casos el modelo logra predecir de forma correcta, existen situaciones en las que clasifica erróneamente noticias con lenguaje ambiguo, sensacionalista o con contenido genérico, lo que impacta en el recall.Finalmente, al evaluar un texto de ejemplo —“El presidente anunció nuevas medidas económicas”—, el sistema lo clasificó como noticia falsa con una confianza de 64.82%. 

<p style="font-size: 16px;">
4. Este resultado ilustra tanto la capacidad del modelo para emitir predicciones sobre nuevos textos como la dificultad que enfrenta para interpretar contextos con baja carga informativa, donde aumenta el riesgo de errores de clasificación.


# **11. Conclusiones**

<p style="font-size: 16px;">
1. Los resultados alcanzados muestran que el modelo constituye una base sólida para la detección automática de noticias falsas en español, dado que logra un nivel de exactitud por encima del 70%, lo cual es competitivo para un prototipo inicial desarrollado con un corpus limitado.

<p style="font-size: 16px;">
2. Se observa que el modelo prioriza la precisión sobre el recall, lo que implica que es más confiable al momento de afirmar que una noticia es falsa, pero a costa de no detectar todas las que realmente lo son. Esto lo convierte en un clasificador conservador, útil en contextos donde los falsos positivos deben minimizarse.

<p style="font-size: 16px;">
3. La brecha entre precisión y recall evidencia la necesidad de mejorar la cobertura del sistema, particularmente para capturar con mayor eficacia noticias falsas que poseen estructuras lingüísticas más diversas o complejas. Esto sugiere que una mayor variedad en los datos de entrenamiento podría beneficiar significativamente su rendimiento.

<p style="font-size: 16px;">
4. El F1-score alcanzado indica que existe un balance moderado entre precisión y recall, pero que aún no es suficiente para un uso en escenarios críticos. Se puede optar por la optimización de hiperparámetros y la experimentación con arquitecturas más avanzadas, como transformers en español.

<p style="font-size: 16px;">
5. El análisis de ejemplos concretos en el DataFrame revela que el modelo tiende a cometer errores en textos cortos, ambiguos o con expresiones genéricas. Esto pone en evidencia la importancia de considerar estrategias de data augmentation, para ampliar la diversidad lingüística en el entrenamiento.

<p style="font-size: 16px;">
6. Si bien el prototipo actual presenta limitaciones, sus resultados confirman el potencial de aplicar técnicas de aprendizaje profundo en la detección de noticias falsas. Con ajustes en los datos, mejoras en la arquitectura y un refinamiento del pipeline.